In [2]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import minimize


In [18]:

def kriging(X, y, x_pred):
    # Calcular la matriz de distancias
    D = squareform(pdist(X))
    
    # Función para calcular la matriz de autocovarianza
    def autocov(theta, D):
        n = D.shape[0]
        A = np.zeros((n, n))
        for i in range(n):
            for j in range(i, n):
                A[i, j] = np.exp(-theta * D[i, j])
                A[j, i] = A[i, j]
        return A
    
    # Función objetivo para minimizar
    def objective(theta):
        A = autocov(theta, D)
        y_pred = np.dot(np.dot(np.linalg.inv(A), y), np.ones(n))
        residuals = y - y_pred
        return np.dot(residuals, residuals)
    
    n = X.shape[0]
    initial_guess = np.ones(n)
    result = minimize(objective, initial_guess)
    theta = result.x
    
    # Calcular la matriz de autocovarianza para los puntos conocidos
    A = autocov(theta, D)
    y_pred = np.dot(np.dot(np.linalg.inv(A), y), np.ones(n))
    
    # Calcular la matriz de covarianza para los puntos conocidos y desconocidos
    D_pred = squareform(pdist(np.vstack((X, x_pred))))
    C = np.zeros((n+1, n+1))
    for i in range(n+1):
        for j in range(i, n+1):
            C[i, j] = np.exp(-theta * D_pred[i, j])
            C[j, i] = C[i, j]
    
    # Calcular la estimación de kriging para el punto desconocido
    y_kriging = np.dot(np.dot(C[:n, n], np.linalg.inv(A)), y_pred)
    
    return y_kriging

    



In [17]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]])
y = np.array([[5], [7], [9], [11], [13]])
x_pred = np.array([[6, 7]])

result = kriging(X, y, x_pred)
print(result)



ValueError: setting an array element with a sequence.